In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    TimestampType,
    StringType,
)

In [19]:
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

In [20]:
spark

In [4]:
spark.sql("SHOW CURRENT NAMESPACE").show()

+-------+----------------+
|catalog|       namespace|
+-------+----------------+
|   demo|retail_analytics|
+-------+----------------+



In [16]:
spark.sql("SHOW TABLES IN demo.retail_analytics").show()

+----------------+------------------+-----------+
|       namespace|         tableName|isTemporary|
+----------------+------------------+-----------+
|retail_analytics|  fact_order_items|      false|
|retail_analytics|      dim_customer|      false|
|retail_analytics|          dim_date|      false|
|retail_analytics|fact_order_summary|      false|
|retail_analytics|       dim_product|      false|
|retail_analytics|        dim_seller|      false|
+----------------+------------------+-----------+



In [21]:
spark.sql("SELECT * FROM dim_date order by date limit 5").show()

+--------+----------+----+-----+---+-----------+------------+-------+-----------+------------+--------------+----------+
|date_key|      date|year|month|day|day_of_week|week_of_year|quarter|day_of_year|is_month_end|is_month_start|is_weekend|
+--------+----------+----+-----+---+-----------+------------+-------+-----------+------------+--------------+----------+
|       1|2020-01-01|2020|    1|  1|          4|           1|      1|          1|       false|          true|     false|
|       2|2020-01-02|2020|    1|  2|          5|           1|      1|          2|       false|         false|     false|
|       3|2020-01-03|2020|    1|  3|          6|           1|      1|          3|       false|         false|     false|
|       4|2020-01-04|2020|    1|  4|          7|           1|      1|          4|       false|         false|      true|
|       5|2020-01-05|2020|    1|  5|          1|           1|      1|          5|       false|         false|      true|
+--------+----------+----+-----+

In [22]:
spark.sql("SELECT * FROM dim_seller order by seller_id limit 5").show()

+--------------------+---------+--------------------+-----+---------------+--------------+------------+----------+
|           seller_sk|seller_id|             address|state|zip_code_prefix|effective_from|effective_to|is_current|
+--------------------+---------+--------------------+-----+---------------+--------------+------------+----------+
|e70248d51bfe39e2e...| 00000001|36475 Landry High...|   SD|          57075|    2021-11-10|        NULL|      true|
|45a4016494e5ce24b...| 00000002|505 Thomas Spring...|   WI|          53489|    2020-06-25|        NULL|      true|
|d34f968c83c2b0239...| 00000003|69257 Stephen Isl...|   TX|          79774|    2020-03-01|        NULL|      true|
|6670b3d01630fdeee...| 00000004| 3070 Jasmine Tunnel|   AK|          99880|    2021-10-20|        NULL|      true|
|ac10fd063621a8f13...| 00000005|78645 Lawrence Cl...|   NM|          87987|    2022-02-08|        NULL|      true|
+--------------------+---------+--------------------+-----+---------------+-----

In [23]:
spark.sql("SELECT * FROM dim_customer order by customer_id limit 5").show()

+--------------------+-----------+--------------------+-----+---------------+--------------+------------+----------+
|         customer_sk|customer_id|             address|state|zip_code_prefix|effective_from|effective_to|is_current|
+--------------------+-----------+--------------------+-----+---------------+--------------+------------+----------+
|da4332b9cfa2205f0...|   00000001|04485 Matthew Por...|   SC|          29120|    2022-09-20|        NULL|      true|
|ed346c335c0b67743...|   00000002|12590 Kristina Forge|   DE|          19892|    2020-06-21|        NULL|      true|
|01cb5a31510a9b634...|   00000003|34324 Nicolas Hei...|   AK|          99662|    2020-02-08|        NULL|      true|
|3cd45099b5adff148...|   00000004|34324 Wolfe Green...|   WA|          98745|    2023-02-25|        NULL|      true|
|8c526b7d7c9ac7f37...|   00000005|872 Stephanie Pra...|   KY|          40865|    2021-03-03|        NULL|      true|
+--------------------+-----------+--------------------+-----+---

In [24]:
spark.sql("SELECT * FROM dim_product order by product_id limit 5").show()

+--------------------+----------+--------------+---------------+--------------+------+-----------+---------+---------+--------+--------------+------------+----------+
|          product_sk|product_id|  product_name|  category_name|  sub_category| price| size_label|length_cm|height_cm|width_cm|effective_from|effective_to|is_current|
+--------------------+----------+--------------+---------------+--------------+------+-----------+---------+---------+--------+--------------+------------+----------+
|5910fc9566bf67f15...|  00000001|Boxify Classic|general_purpose|Boxify Classic| 35.20|      Small|     25.5|     15.5|    20.5|    2020-01-11|        NULL|      true|
|bcb50ebf45d8c4857...|  00000002|Boxify Classic|general_purpose|Boxify Classic| 60.50|     Medium|     35.5|     20.5|    30.5|    2020-01-11|        NULL|      true|
|3c07d06fac370e1fb...|  00000003|Boxify Classic|general_purpose|Boxify Classic| 85.80|      Large|     50.5|     30.5|    40.5|    2020-01-11|        NULL|      true

In [25]:
spark.sql("SELECT * FROM fact_order_summary order by order_summary_sk limit 5").show()

+----------------+--------------------+--------+------------+------------------+-------------------+----------+--------------+-----------------------+-----------------------------+------------------------------+---------------------------+---------------------------+---------------------------+
|order_summary_sk|         customer_sk|order_id|order_status| total_order_value|total_freight_value|item_count|order_date_key|order_approved_date_key|delivered_to_carrier_date_key|delivered_to_customer_date_key|estimated_delivery_date_key|min_shipping_limit_date_key|max_shipping_limit_date_key|
+----------------+--------------------+--------+------------+------------------+-------------------+----------+--------------+-----------------------+-----------------------------+------------------------------+---------------------------+---------------------------+---------------------------+
|               0|3250891df5693202f...|00000004|  PROCESSING|             149.9| 19.020000000000003|         3| 

In [26]:
spark.sql("SELECT * FROM fact_order_items order by order_item_sk limit 5").show()

+-------------+--------------------+--------------------+--------+-------------+----------+-------------+--------------+-----------------------+
|order_item_sk|           seller_sk|          product_sk|order_id|order_item_id|item_value|freight_value|order_date_key|shipping_limit_date_key|
+-------------+--------------------+--------------------+--------+-------------+----------+-------------+--------------+-----------------------+
|            0|129df2595643bd622...|39f85669cc8ca96d2...|00000003|            0|     117.0|        34.57|          1329|                   NULL|
|            1|0d76d83808ec4582c...|67ad7d05384608ba3...|00000004|            1|      71.5|          7.4|          1280|                   NULL|
|            2|45f938901ae650f60...|e96c80f74414db805...|00000007|            2|     138.0|        40.11|          1372|                   NULL|
|            3|a8f10e03e3da01597...|f5fa40c162fa53228...|00000010|            1|     128.8|        37.89|          1150|          

In [18]:
spark.stop()